In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as mpl
from datetime import timedelta

from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

import plotly.graph_objects as go
from plotly.subplots import make_subplots

colors = ["#e6194b", "#3cb44b", "#ffe119", "#0082c8", "#f58231", "#911eb4", "#46f0f0", "#f032e6", "#d2f53c", "#fabebe", "#008080", 
          "#e6beff", "#aa6e28", "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1", "#000080", "#808080", "#ffffff", "#000000"]

In [ ]:
actions_suffix = '_actions.csv'
value_suffix = '_state.csv'

In [ ]:
def get_dfs(names, model, stat_files):
    dfs = []
    for idx, name in enumerate(names):
        strategy_name = f'{model}/{stat_files[idx]}/all'
        portfolio_prefix = f'../results/cs_eval/{strategy_name}'

        df = pd.read_csv(f"{portfolio_prefix}{value_suffix}")  # , index_col=0
        df = df.rename(columns={df.columns[0]: 'date'})
        df.loc[0, 'date'] = pd.to_datetime(df.loc[1]['date']) - timedelta(days=1)
        df['date'] = pd.to_datetime(df['date'])
        df['return'] = df['account_value'].pct_change() + 1
        df['equity'] = (df['return']).cumprod()
        df.loc[0, 'return'] = 1
        df.loc[0, 'equity'] = 1
        df.index = df['date']
        df = df.drop('date', axis=1)

        df2 = pd.read_csv(f"{portfolio_prefix}{actions_suffix}", index_col=0)
        df2.index = pd.to_datetime(df2.index)

        df['strategy'] = df['return']
        df_strategy = df[['strategy']]         
        df_returns = df_strategy.cumprod()
        dfs.append(df_returns)
    return dfs

def get_benchmark(dfs):
    benchmark_returns_1d = '../datasets/thesis/benchmark/1h_benchmark_returns.csv'
    df_benchmark = pd.read_csv(f"{benchmark_returns_1d}")
    df_benchmark.index = pd.to_datetime(df_benchmark['date'])
    df_benchmark = df_benchmark.drop('date', axis=1)
    df_benchmark = df_benchmark[(df_benchmark.index >= dfs[0].index[0]) & (df_benchmark.index <= dfs[0].index[-1])]
    df_benchmark = df_benchmark.cumprod()
    return df_benchmark

def get_prices(sample_strat):
    stock_prices = '../datasets/thesis/1d/all_1d.csv'
    df3 = pd.read_csv(stock_prices)
    df3['date'] = pd.to_datetime(df3['date'])
    # get or column with close price for each stock per tick
    df3 = df3.pivot(index="date", columns="tic", values="close")
    # limit dataset to portfolio range
    df3 = df3[(df3.index >= sample_strat.index[0]) & (df3.index <= sample_strat.index[-1])]
    # get relative change from price
    df3 = df3.pct_change()
    # set first row to 0
    df3.loc[df3.iloc[0].name] = 0
    # get cumulative return for each stock
    df3 = (df3 + 1).cumprod()
    return df3

def plot_performance(model, names, strats, benchmarks, prices, plot_folder, show_prices=True, save=True):    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if show_prices:
        for idx, name in enumerate(prices.columns):
            fig.add_trace(go.Scatter(x=prices.index,y=prices[name],name=name, marker={'color':'#fff'}),secondary_y=False)

    for idx, name in enumerate(names):
        cdf = strats[idx]
        fig.add_trace(go.Scatter(x=cdf.index,y=cdf['strategy'],name=name),secondary_y=False)

    # benchmarks = dfs[0][["CRP", "BAH", "MPT"]]
    cs = ['#8A9A5B', '#36454F', '#A9A9A9']
    for idx, name in enumerate(benchmarks.columns):
        fig.add_trace(go.Scatter(x=benchmarks.index,y=benchmarks[name],name=name, marker={'color':cs[idx]}),secondary_y=False)

    fig.update_layout(title_text=f'{model} Strategy Performance compared to benchmarks', height=600)
    fig.show()
    if save:
        fig.write_html(f"{plot_folder}/performance_compare_{model}.html")

In [ ]:
names = ["V213", "V219", "V221"]
model = "A2C"
stat_files = ["EVAL_A2C_V213_12201326_200M", "EVAL_A2C_V219_12201317_200M", "EVAL_A2C_V221_12200914_200M"]

dfs_a2c = get_dfs(names, model, stat_files)
df_benchmark = get_benchmark(dfs_a2c)
df_prices = get_prices(dfs_a2c[0])

plot_folder = f'../results/cs_eval/{model}'
plot_performance(model, names, dfs_a2c, df_benchmark, df_prices, plot_folder)

In [ ]:
names = ["V205", "V207", "V208"]
model = "PPO"
stat_files = ["EVAL_PPO_V205_12201259_120M", "EVAL_PPO_V207_12200935_120M", "EVAL_PPO_V208_12200925_120M"]

dfs_ppo = get_dfs(names, model, stat_files)
df_benchmark = get_benchmark(dfs_ppo)
df_prices = get_prices(dfs_ppo[0])

plot_folder = f'../results/cs_eval/{model}'
plot_performance(model, names, dfs_ppo, df_benchmark, df_prices, plot_folder)

In [ ]:
names = ["V203", "V213", "V214"]
model = "TD3"
stat_files = ["EVAL_TD3_V203_12201307_12M", "EVAL_TD3_V213_12201319_12M", "EVAL_TD3_V214_12201313_12M"]

dfs_td3 = get_dfs(names, model, stat_files)
df_benchmark = get_benchmark(dfs_td3)
df_prices = get_prices(dfs_td3[0])

plot_folder = f'../results/cs_eval/{model}'
plot_performance(model, names, dfs_td3, df_benchmark, df_prices, plot_folder)

In [ ]:
best = ["EVAL_A2C_V221_12200914_200M", "EVAL_PPO_V208_12200925_120M", "EVAL_TD3_V214_12201313_12M"]
dfs2 = []
dfs2.append(dfs_a2c[2])
dfs2.append(dfs_ppo[2])
dfs2.append(dfs_td3[2])
names = ["A2C", "PPO", "TD3"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/cs_eval/'
model = "1D"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False)


In [ ]:
cfg = {
    "A2C": ["EVAL_A2C_V221_12200914_200M"],
    "PPO": ["EVAL_PPO_V208_01121911_1D_180M"],
    "TD3": ["EVAL_TD3_V214_12201313_12M"],    
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])
names = ["A2C", "PPO", "TD3"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/cs_eval/'
model = "1D"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)


In [ ]:
cfg = {
    "A2C": ["EVAL_A2C_V221_01121916_12H_600M"],
    "PPO": ["EVAL_PPO_V208_01121435_12H_600M"],
    "TD3": ["EVAL_TD3_V214_01130958_12H_54M"],    
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])
names = ["A2C", "PPO", "TD3"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/cs_eval/'
model = "12H"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)


In [ ]:
cfg = {
    "A2C": ["EVAL_A2C_V221_01121914_6H_550M"],
    "PPO": ["EVAL_PPO_V208_01121514_6H_550M"],
    "TD3": ["EVAL_TD3_V214_01130933_6H_54M"],    
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])
names = ["A2C", "PPO", "TD3"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/cs_eval/'
model = "6H"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)


In [ ]:
cfg = {
    "A2C": ["EVAL_A2C_V221_01122308_1H_240M"],
    "PPO": ["EVAL_PPO_V208_01121518_1H_200M"],
    "TD3": ["EVAL_TD3_V214_01122319_1H_42M"],    
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])
names = ["A2C", "PPO", "TD3"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/cs_eval/'
model = "1H"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)


In [ ]:
cfg = {
    "A2C": ["EVAL_A2C_V221_01122308_1H_240M", "EVAL_A2C_V221_01121914_6H_550M", "EVAL_A2C_V221_01121916_12H_600M", "EVAL_A2C_V221_12200914_200M"]
}

names = ["1H", "6H", "12H", "1D"]
dfs = get_dfs(names, "A2C", cfg["A2C"])


df_benchmark = get_benchmark(dfs)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/cs_eval/'
model = "All A2C"
plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)


In [ ]:
cfg = {
    "PPO": ["EVAL_PPO_V208_01121518_1H_200M", "EVAL_PPO_V208_01121514_6H_550M", "EVAL_PPO_V208_01121435_12H_600M", "EVAL_PPO_V208_01121911_1D_180M"]
}

names = ["1H", "6H", "12H", "1D"]
dfs = get_dfs(names, "PPO", cfg["PPO"])


df_benchmark = get_benchmark(dfs)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/cs_eval/'
model = "All PPO"
plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)

In [ ]:
cfg = {
    "TD3": ["EVAL_TD3_V214_01122319_1H_42M", "EVAL_TD3_V214_01130933_6H_54M", "EVAL_TD3_V214_01130958_12H_54M", "EVAL_TD3_V214_12201313_12M"]
}

names = ["1H", "6H", "12H", "1D"]
dfs = get_dfs(names, "TD3", cfg["TD3"])


df_benchmark = get_benchmark(dfs)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/cs_eval/'
model = "All TD3"
plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)

In [ ]:
cfg = {
    "A2C": ["EVAL_A2C_V221_01121916_12H_600M"],
    "PPO": ["EVAL_PPO_V208_01121435_12H_600M"],
    "TD3": ["EVAL_TD3_V214_01130958_12H_54M"],    
}

dfs2 = []
dfs_a2c = get_dfs(["A2C"], "A2C", cfg["A2C"])
dfs_ppo = get_dfs(["PPO"], "PPO", cfg["PPO"])
dfs_td3 = get_dfs(["TD3"], "TD3", cfg["TD3"])


dfs2.append(dfs_a2c[0])
dfs2.append(dfs_ppo[0])
dfs2.append(dfs_td3[0])
names = ["A2C", "PPO", "TD3"]
df_benchmark = get_benchmark(dfs2)
df_prices = get_prices(dfs2[0])

plot_folder = f'../results/cs_eval/'
model = "Short 12H"
plot_performance(model, names, dfs2, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)
